In [26]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "HardcoreColtsFan$"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64, {}'.format(encoded_image.decode()),
                         style={'height':'100px'})),
    html.Center(html.H1('Grazioso Salvare Dashboard - Chase Knoblock')),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Indivdual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '15px'}
        )
    ]),
    
    html.Hr(),
    
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'font_family': 'Arial', 'padding': '5px'},
        style_header={'backgroundColor': '#add8e6', 'fontWeight': 'bold'}
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        dff = pd.DataFrame.from_records(db.read({
            'animal_type': 'Dog', 
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}
        }))
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = pd.DataFrame.from_records(db.read({
            'animal_type': 'Dog', 
            'breed':{'$in': ['German Shepherd', 'Alaskan Malamute', 'Siberian Husky', 'Rottweiler']}
        }))
    elif filter_type == 'Disaster or Individual Tracking':
        dff = pd.DataFrame.from_records(db.read({
            'animal_type': 'Dog', 
            'breed':{'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}
        }))
    else:
        dff = pd.DataFrame.from_records(db.read({}))
    
    dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame.from_dict(viewData)
    
    fig = px.pie(dff, names='breed', title='Preferred Breeds in Current Filter')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return [html.Center(html.P("Select an animal from the table to view location."))]
    
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    try:
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]
    except Exception:
        lat = dff.get('lat') or dff.get('latitude') or dff.get('Latitude') or None
        lon = dff.get('lon') or dff.get('longitude') or dff.get('Longitude') or None
        if isinstance(lat, pd.Series):
            lat = lat.iloc[row]
        if isinstance(lon, pd.Series):
            lon = lon.iloc[row]
    
    try:
        breed = dff.iloc[row, 4]
    except Exception:
        breed = dff.get('breed') or dff.get('animal_breed') or dff.get('Breed') or 'Unknown'
    
    try: name = dff.iloc[row, 9]
    except Exception:
        name = dff.get('name') or dff.get('animal_name') or dff.get('Name') or 'No name'
    
    if lat is None or lon is None:
        return html.Div("Selected row does not contain usable latitutde/longitude.")
                    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
            center=[float(lat), float(lon)], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id",
                            url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
                            attribution="© OpenStreetMap contributors"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(
                    position=[float(lat), float(lon)], 
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(name))
                        ])
                    ]
                )
            ]
        )
    ]
            

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://passagerufus-finalkiwi-3000.codio.io/proxy/8050/
